#  *** DRAFT ***
##### Author: Omer Nivron
##### Date: 04/09/19
#####  Data source: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package 

In [25]:
from aus_rain_cleanup import *
from aus_rain_feat_eng import *
from aus_rain_modeling import *
import seaborn as sns

# Report - Australian rain prediction 

### Results:    ~85% success rate in predicting tomorrow's rain outcome (still plenty potentail)
####                      - The most important features to predict rain are locations and wind directions. Whether it rained today doesn't seem very important from the model (however it might just be correlated with another variable)

### Assumptions
 - Missing values are reasonably replaced by mode and median per month and location for categorical and numeric columns respictively 
 - It is valid to drop columns with missing value share greater than 0.35
 - Inheirt Tree model assumptions

### Open issues/ problems

- Model is overfitting: learning can't generalize well to new data  

### Analysis method explained

The analysis included cleaning data with the main issue being plenty missing values; Grouping by year, month and location, the median was used for numeric columns since it's more reliable when data isn't symmetric; The mode was used for categorical columns. Other cleaning procedures included formating (e.g. string to date). I then transformed categorical variables to one-hot encodings since the model can't handle strings. Then I split the data to training data (everything prior to 2015) and test data that was never seen by the model (2015 onwards) and is used to asses results on unseen data. After all data was model ready I ran it once with a decision tree model and a random forest model. Currently our metric is misclassification rate: how many times our model got it wrong divided by size of data (this metric might be misleading and future work should consider other options)   

##### Why did I choose the tree based models for the task ? 

- Can deal better than other models with missing values (depending implementation)
- No need to normalize data or make any transformations
- Quick predictions 
- Robust to outliers
- Easy interpretability (variable importance and selection)
- Empirically strong results 

### Run analysis

In [26]:
australian_rain_df = pd.read_csv('./weatherAUS.csv')
####### Cleanup #######
cat_cols, num_cols, australian_rain_df = aus_rain_cleanup.cleanup(australian_rain_df)
######
###### Feature engineering ######
australian_rain_df = transform_to_dummies(australian_rain_df, list(cat_cols) + ['Location'])
######
###### Modelling ######
feat_importance, results_pd = model()
######

AttributeError: module 'aus_rain_cleanup' has no attribute 'cleanup'

### Models' results 

In [ ]:
results_pd.reset_index(drop=True)

In [27]:
loc = 'Adelaide'
num_var = 'WindSpeed3pm'

In [28]:
australian_rain_df[australian_rain_df['Location'] == loc].boxplot(column=num_var, by =['Month'])

KeyError: 'Month'

### To do's 

 - Exploration:
1. Check for areas where it never rains and drop from dataset 
2. Check the correlations of variables with rain today status 
 
        
 - Model : 
1. Add a baseline model for comparison 
2. Hyperparameter optimization
3. Consider logistic regression model
4. Balance data (sampling)
5. Add true-pos and false-pos metrics
6. Add ROC curve
